In [15]:
import pandas as pd
import csv

import os, os.path
import sys

# to move files from one directory to another
import shutil

In [16]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
plots_dir = "/Users/hn/Documents/01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/"

In [17]:
# choices_set_1 = pd.read_excel(io=param_dir + "set1_PerryandCo.xlsx", sheet_name=0)
# response_set_1 = pd.read_excel(io=param_dir + "Perry_and_Co_Responses_set_1.xlsx", sheet_name=0)

In [18]:
choices_set_1_xl = pd.ExcelFile(perry_dir + "set1_PerryandCo.xlsx")
choises_set_1_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names

response_set_1_xl = pd.ExcelFile(perry_dir + "Perry_and_Co_Responses_set_1.xlsx")
response_set_1_sheet_names = response_set_1_xl.sheet_names  # see all sheet names


In [19]:
print (choises_set_1_sheet_names)
print (response_set_1_sheet_names)

['extended_1', 'extended_2', 'extended_3', 'extended_4', 'extended_5', 'extended_6']
['S1 - F6', 'S1 - F5', 'S1 - F4', 'S1 - F3', 'S1 - F2', 'S1 - F1']


In [20]:
a_choice_sheet = choices_set_1_xl.parse(choises_set_1_sheet_names[0])
a_response_sheet = response_set_1_xl.parse(response_set_1_sheet_names[0])

In [21]:
print (list(a_choice_sheet.columns))

['Question_in_set', 'Question_overall', 'ID', 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', 'latitude', 'longitude', 'CropTyp', 'Irrigtn', 'DataSrc', 'Acres', 'ExctAcr', 'LstSrvD', 'county']


# List Crop Types in Set 1

In [22]:
crop_types = []

for a_choice_sheet in choises_set_1_sheet_names:
    
    # read a damn sheet
    a_choice_sheet = choices_set_1_xl.parse(a_choice_sheet)
    unique_crop_types = a_choice_sheet['CropTyp'].unique()

    # add them to the damn list
    crop_types += list(unique_crop_types)

In [23]:
print (crop_types)
print ()
print ("There are [{cro_type_count}] crop types.".format(cro_type_count=len(crop_types)))

['corn, field', 'corn, sweet', 'corn seed', 'wheat', 'wheat fallow', 'potato', 'potato', 'bean, dry', 'bean, green', 'onion', 'pea seed', 'pea, dry', 'pea, green', 'barley', 'barley hay', 'buckwheat', 'buckwheat', 'canola', 'carrot', 'carrot seed', 'market crops', 'mint', 'mint', 'oat hay', 'triticale', 'triticale hay', 'yellow mustard', 'alfalfa seed', 'alfalfa seed', 'grass seed', 'bluegrass seed', 'grass hay']

There are [32] crop types.


## Count number of questions

In [24]:
question_count = 0

for a_choice_sheet in choises_set_1_sheet_names:
    
    # read a damn sheet
    a_choice_sheet = choices_set_1_xl.parse(a_choice_sheet)

    # add them to the damn list
    question_count += a_choice_sheet.shape[0]

print('There are [{ques_count}] questions.'.format(ques_count=question_count))

There are [294] questions.


In [25]:
unwanted_opinions = ["kirtir@wsu.edu"]

In [26]:
a_response_sheet

,Timestamp,Email Address,"QUESTION 1 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.03408928,-118.6901155/@46.03408928,-118.6901155,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?,Comments,"QUESTION 2 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.0291209,-118.73977856/@46.0291209,-118.73977856,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.1,Comments.1,"QUESTION 3 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.00323746,-118.6146822/@46.00323746,-118.6146822,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.2,...,Comments.40,"QUESTION 42 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/47.1957135,-119.93708271/@47.1957135,-119.93708271,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.41,Comments.41,"QUESTION 43 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.04293613,-118.55966589/@46.04293613,-118.55966589,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.42,Comments.42,"QUESTION 44 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.03868929,-118.49344599/@46.03868929,-118.49344599,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.43,Comments.43
0,2022-02-02 11:57:26.602,andrew.mcguire@wsu.edu,Single Crop,NaN,NaN,Single Crop,NaN,NaN,Single Crop,NaN,...,NaN,Single Crop,NaN,NaN,Single Crop,NaN,NaN,Single Crop,NaN,NaN
1,2022-02-03 15:47:12.553,pbeale@agr.wa.gov,Single Crop,NaN,NaN,Single Crop,NaN,NaN,Single Crop,NaN,...,NaN,Double Crop,NaN,This looks like late buckwheat double cropped ...,Single Crop,NaN,NaN,Single Crop,NaN,NaN
2,2022-02-04 10:36:02.818,twaters@wsu.edu,Single Crop,NaN,NaN,Single Crop,NaN,NaN,Single Crop,NaN,...,NaN,Single Crop,NaN,NaN,Single Crop,NaN,NaN,Single Crop,NaN,NaN


In [61]:
## Define the damn output dataframe

vote_df = pd.DataFrame(columns=['Form', 'Question', 'opinion_count', 'ID', 
                                "Perry", "Andrew", "Tim", "Kirti"], 
                         index=range(question_count))
vote_df.head(1)
curr_row = 0

extended_choices = pd.DataFrame()

###### populate the output datafrme

for response_sheet_name in response_set_1_sheet_names:
    
    # pick up the numeric part of the sheet names from google forms sheets
    # this is the form number as well.
    sheet_numeric_part = [s for s in response_sheet_name.split()[-1] if s.isdigit()]
    sheet_numeric_part = "".join(sheet_numeric_part)
    
    # form sheet names of choices excel sheets
    choise_sheet_name = "extended_" + sheet_numeric_part
    
    a_choice_sheet = choices_set_1_xl.parse(choise_sheet_name)
    a_response_sheet = response_set_1_xl.parse(response_sheet_name)
    
    extended_choices = pd.concat([extended_choices, a_choice_sheet])
    #
    # drop unwanted opinions, i.e. keep experts opinions
    #
    # a_response_sheet = a_response_sheet[~(a_response_sheet["Email Address"].isin(unwanted_opinions))].copy()
    
    for a_col_name in a_response_sheet.columns:
        if "QUESTION" in a_col_name:
            question_number = a_col_name.split()[1]
            
            vote_df.loc[curr_row, "Form"] = sheet_numeric_part
            vote_df.loc[curr_row, "Question"] = question_number
            vote_df.loc[curr_row, "opinion_count"] = len(a_response_sheet[a_col_name].unique())
            vote_df.loc[curr_row, "ID"] = a_choice_sheet.loc[int(question_number)-1, "ID"]
            
             
            vote_df.loc[curr_row, "Perry"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "pbeale@agr.wa.gov"][a_col_name].values[0]
            
            vote_df.loc[curr_row, "Andrew"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "andrew.mcguire@wsu.edu"][a_col_name].values[0]
            
            vote_df.loc[curr_row, "Tim"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "twaters@wsu.edu"][a_col_name].values[0]
            
            if "kirtir@wsu.edu" in list(a_response_sheet['Email Address']):
                vote_df.loc[curr_row, "Kirti"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "kirtir@wsu.edu"][a_col_name].values[0]
            
            curr_row += 1
            
            
out_name = perry_dir + "set1_all_votes.csv"
vote_df.to_csv(out_name, index = False)

In [65]:
vote_df.Kirti.unique()

array([nan, 'Single Crop', 'Unsure', 'Double Crop', 'Mustard Crop'],
      dtype=object)

In [59]:
## Define the damn output dataframe

vote_df = pd.DataFrame(columns=['Form', 'Question', 'opinion_count', 'ID', "Perry", "Andrew", "Tim"], 
                       index=range(question_count))
vote_df.head(1)
curr_row = 0

extended_choices = pd.DataFrame()

###### populate the output datafrme

for response_sheet_name in response_set_1_sheet_names:
    
    # pick up the numeric part of the sheet names from google forms sheets
    # this is the form number as well.
    sheet_numeric_part = [s for s in response_sheet_name.split()[-1] if s.isdigit()]
    sheet_numeric_part = "".join(sheet_numeric_part)
    
    # form sheet names of choices excel sheets
    choise_sheet_name = "extended_" + sheet_numeric_part
    
    a_choice_sheet = choices_set_1_xl.parse(choise_sheet_name)
    a_response_sheet = response_set_1_xl.parse(response_sheet_name)
    
    extended_choices = pd.concat([extended_choices, a_choice_sheet])
    #
    # drop unwanted opinions, i.e. keep experts opinions
    #
    a_response_sheet = a_response_sheet[~(a_response_sheet["Email Address"].isin(unwanted_opinions))].copy()
    
    for a_col_name in a_response_sheet.columns:
        if "QUESTION" in a_col_name:
            question_number = a_col_name.split()[1]
            
            vote_df.loc[curr_row, "Form"] = sheet_numeric_part
            vote_df.loc[curr_row, "Question"] = question_number
            vote_df.loc[curr_row, "opinion_count"] = len(a_response_sheet[a_col_name].unique())
            vote_df.loc[curr_row, "ID"] = a_choice_sheet.loc[int(question_number)-1, "ID"]
            
             
            vote_df.loc[curr_row, "Perry"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                              "pbeale@agr.wa.gov"][a_col_name].values[0]
            
            vote_df.loc[curr_row, "Andrew"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                               "andrew.mcguire@wsu.edu"][a_col_name].values[0]
            
            vote_df.loc[curr_row, "Tim"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                            "twaters@wsu.edu"][a_col_name].values[0]
            
            curr_row += 1

In [14]:
extended_output = pd.merge(vote_df, extended_choices, on=['ID'], how='left')
extended_output.head(2)


,Form,Question,opinion_count,ID,Perry,Andrew,Tim,Question_in_set,Question_overall,NDVI_TS_Name,...,TOA_RGB,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,6,1,1,157415_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,1,251,Walla2015_157415_WSDA_SF_2015.png,...,157415_WSDA_SF_2015_46.03408928_-118.6901155_T...,46.034089,-118.690116,alfalfa seed,sprinkler,wsda,28,27.675520,2015/06/04 00:00:00,Walla Walla
1,6,2,1,155601_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,2,252,Walla2015_155601_WSDA_SF_2015.png,...,155601_WSDA_SF_2015_46.0291209_-118.73977856_T...,46.029121,-118.739779,alfalfa seed,sprinkler,wsda,111,110.768754,2015/06/04 00:00:00,Walla Walla


In [15]:
# check of the question numbers are correct.
sum(extended_output['Question'].astype(int) - extended_output['Question_in_set'])

0

In [16]:
# Total number of disagreements

disagreement_table = extended_output[extended_output.opinion_count != 1].copy()
disagreement_table.shape

(120, 21)

In [17]:
disagreement_table = disagreement_table[disagreement_table.CropTyp != "grass hay"].copy()

In [18]:
disagreement_table.shape

(118, 21)

In [19]:
disagreement_table.reset_index(inplace=True, drop=True)
disagreement_table_grouped = disagreement_table.groupby(['CropTyp', 'opinion_count']).count()
disagreement_table_grouped.reset_index(inplace=True, drop=True)
disagreement_table_grouped.sort_values(by=["opinion_count", 'ID', "CropTyp"],
                                    ascending = [False, False, True],
                                    inplace=True)

print (list(disagreement_table_grouped[disagreement_table_grouped.opinion_count==3].CropTyp.unique()))

KeyError: 'opinion_count'

In [ ]:
disagreement_table_grouped.head(6)

In [ ]:
disagreement_table_grouped.shape

## Group by crop type and count number of disagreements

In [ ]:
extended_output_grouped = extended_output.groupby(['CropTyp', 'opinion_count']).count()
extended_output_grouped.reset_index(inplace=True, drop=True)
extended_output_grouped.sort_values(by=['opinion_count', "CropTyp"],
                                    ascending = [False, True],
                                    inplace=True)

extended_output_grouped.head(2)

In [ ]:
extended_output_grouped.shape

In [ ]:
print (list(extended_output_grouped[extended_output_grouped.opinion_count==3].CropTyp.unique()))

In [ ]:
extended_output_grouped = extended_output.groupby(['CropTyp', 'opinion_count']).count()
extended_output_grouped.reset_index(inplace=True, drop=True)
extended_output_grouped.sort_values(by=['ID', "opinion_count",  "CropTyp"],
                                    ascending = [False, True, True],
                                    inplace=True)

print (list(extended_output_grouped[extended_output_grouped.opinion_count==3].CropTyp.unique()))

In [ ]:
extended_output_grouped

### Export disagreement table of set 1

In [ ]:
output_dir = perry_dir
disagreement_table.sort_values(by=["ID", 'opinion_count', "CropTyp"],
                                    ascending = [False, False, True],
                                    inplace=True)

out_name = output_dir + "disagreement_table_set_1_sortCropOpinion.csv"
disagreement_table.to_csv(out_name, index = False)


disagreement_table.sort_values(by=["ID", 'opinion_count', "CropTyp"],
                                    ascending = [False, False, True],
                                    inplace=True)


out_name = output_dir + "disagreement_table_set_1_sortOpinionCrop.csv"
disagreement_table.to_csv(out_name, index = False)


##########
########## all stats
##########
extended_output.sort_values(by=['opinion_count', "CropTyp"],
                            ascending = [False, True],
                            inplace=True)

out_name = output_dir + "set_1_experts_stats_extended_sortOpinionCrop.csv"
extended_output.to_csv(out_name, index = False)

extended_output.sort_values(by=["CropTyp", "opinion_count"],
                            ascending = [True, False],
                            inplace=True)

out_name = output_dir + "set_1_experts_stats_extended_sortCropOpinion.csv"
extended_output.to_csv(out_name, index = False)

### Create a long PDF of fields where disagreements occur

In [ ]:
import subprocess

In [ ]:
os.getcwd()

In [ ]:

# with open("sometexfile.tex","w") as file:
#     file.write("\\documentclass{article}\n")
#     file.write("\\begin{document}\n")
#     file.write("Hello Palo Alto!\n")
#     file.write("\\end{document}\n")



In [ ]:
# import subprocess, os

# with open("sometexfile.tex","w") as file:
#     file.write("\\documentclass{article}\n")
#     file.write("\\begin{document}\n")
#     file.write("Hello Palo Alto!\n")
#     file.write("\\end{document}\n")
    
#     x = subprocess.call("pdflatex sometexfile.tex")
#     if x != 0:
#         print("Exit-code not 0, check result!")
#     else:
#         os.system("start sometexfile.pdf")

In [ ]:
disagreement_table.sort_values(by=["CropTyp"],
                                   ascending = [True],
                                   inplace=True)
disagreement_table.reset_index(inplace=True, drop=True)
disagreement_table.head(2)

In [ ]:
disagreement_table.sort_values(by=['CropTyp'],
                           ascending = [True],
                           inplace=True)
disagreement_table.reset_index(inplace=True, drop=True)

os.chdir('/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/PDF_set_1_disagreement/')
with open("PDF_set_1_disagreement.tex","w") as file:
    file.write("\\documentclass{article}\n")

    # file.write("\\usepackage[utf8]{inputenc}\n")
    # file.write("\\usepackage[scaled=0.85]{beramono}\n")
    # file.write("\\pdfoutput=24\n")
    # file.write("\\usepackage{framed, color}\n")
#    file.write("\\usepackage{etaremune}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")

#     file.write("\\definecolor{mgreen}{RGB}{25,147,100}\n")
#     file.write("\\definecolor{shadecolor}{rgb}{1,.8,.1}\n")
#     file.write("\\definecolor{shadecolor2}{RGB}{245,237,0}\n")
#     file.write("\\definecolor{orange}{RGB}{255,137,20}\n")
#     file.write("\\definecolor{orange}{RGB}{245,37,100}\n")

#     file.write("\\usepackage[english]{babel}\n")
#     file.write("\\usepackage{babel,blindtext}\n")
#     file.write("\\usepackage{fullpage}\n")
#     file.write("\\usepackage{amsfonts}\n")
#     file.write("\\usepackage{lscape}\n")
#     file.write("\\usepackage{bbm}\n")
#     file.write("\\usepackage{todonotes}\n")
#     file.write("\\usepackage{cite}\n")
#     file.write("\\usepackage{verbatim}\n")
#     file.write("\\usepackage{bm}\n")

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
#     file.write("\\usepackage{authblk}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

#     file.write("\\newcommand{\\blue}{\\color{blue}}\n")
#    file.write("\\newcommand*{\\affaddr}[1]{#1}\n")
#    file.write("\\newcommand*{\\affmark}[1][*]{\\textsuperscript{#1}}\n")

    file.write("\\title{\\bf Set 1 - Disagreements}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")


    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    file.write("\\section{Disagreements}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in range(disagreement_table.shape[0]):
        curr_row = disagreement_table.loc[a_row]
        TS_file_name = curr_row["NDVI_TS_Name"]
        file.write("\\begin{figure*}[ht]\n")
        file.write("\\centering\n")
        file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                     "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                         TS_file_name + "}\n")
        file.write("\\caption[]{" + \
                   "\\textbf{\\color{red}{" + curr_row["CropTyp"] + "}}" + \
                   ", ID: " + curr_row["ID"].replace("_", "\_") + \
                   ", Form: " + curr_row["Form"] + \
                   ", Question: " + curr_row["Question"] + "." + \
                   "\\\\\\hspace{\\textwidth} Perry: " + curr_row["Perry"] + \
                   "\\\\\\hspace{\\textwidth} Andrew: " + curr_row["Andrew"] + \
                   "\\\\\\hspace{\\textwidth} Tim: " + curr_row["Tim"] + \
                   "}\n")
        file.write("\\label{fig:figure" + str(a_row) + "}\n")
        file.write("\\end{figure*}\n")
        
        file.write("\\clearpage\n")
        
        file.write("\n")
        file.write("\n")
        file.write("\n")

    file.write("\\end{document}\n")